# Análise exploratória

## Carregamento e preparação dos dados

### Obtenção e carregamento de dados  

In [ ]:
#importação de bibliotecas
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
# Importação dos dados
df_dataset = pd.read_csv('../Data/Normalization/Course/data_course_normalized.csv', sep=';', encoding='utf-8-sig')

# Carregar dicionário de dados
df_dictionary = pd.read_csv("../Data/Original/dictionary.csv", encoding="ISO-8859-1", sep=";", on_bad_lines='skip')

### Informações sobre o dataset

In [ ]:
#Exibir informações sobre o dataframe
df_dataset.info()

#Exibir as primeiras linhas do dataframe
df_dataset.describe()

### Visualização inicial dos dados

In [ ]:
#Exibir as primeiras linhas do dataframe
df_dataset.head()

### Definição de índices

In [ ]:
# Colocar a coluna 'survey_Id' na primeira posição
df_dataset = df_dataset[['survey_Id'] + [col for col in df_dataset.columns if col != 'survey_Id']]

# Definir o índice do dataframe como a coluna 'survey_Id'
df_dataset.set_index('survey_Id', inplace=True)

# Exibir as primeiras linhas do dataframe
df_dataset.head()

## Análise de variáveis do tipo texto

In [ ]:
# Mapeamentos
mapa_perguntas = dict(zip(df_dictionary["Variable"], df_dictionary["Question"]))
mapa_areas     = dict(zip(df_dictionary["Variable"], df_dictionary["Details"]))

# Estilo Seaborn sem grid
sns.set_theme(style="white")

# Parâmetros de controle
LIMITE_CATEGORIAS_VERTICAIS = 10   # a partir de quantas categorias usar horizontal
TOP_N                       = 30   # máximo de categorias a exibir
LIMITE_ALTURA               = 0.4  # polegadas por categoria
ALTURA_MAXIMA               = 30   # altura máxima em polegadas

# Seleciona somente as colunas
text_columns = df_dataset[['D9_3_1_college_area_description','D9_3_2_master_area_description','D9_3_3_doctorate_area_description','D9_3_4_postdoc_area_description']]

for col in text_columns:
    # ——> Remove NaN antes de contar
    serie = df_dataset[col].dropna()

    # ——> Atribui o valor NA para valores NaN 
    #serie = df_dataset[col].fillna("NA").astype(str)

    # cálculo de Frecuencia e percentual (sem NaN)
    freq = serie.value_counts()
    pct  = serie.value_counts(normalize=True) * 100

    freq_df = pd.DataFrame({
        "Valor":          freq.index.astype(str),
        "Frecuencia":     freq.values,
        "Porcentaje (%)": pct.values
    })

    # verifica truncamento
    truncado = False
    if len(freq_df) > TOP_N:
        freq_df = freq_df.head(TOP_N)
        truncado = True

    # títulos
    titulo    = col
    subtitulo = mapa_perguntas.get(col, "")
    area      = mapa_areas.get(col, "")

    # cálculo de altura (limitada)
    altura = min(LIMITE_ALTURA * len(freq_df) + 3, ALTURA_MAXIMA)
    fig, ax = plt.subplots(figsize=(12, altura))
    fig.subplots_adjust(left=0.5, right=0.95)

    # lógica de orientação
    if len(freq_df) > LIMITE_CATEGORIAS_VERTICAIS:
        # barras horizontais
        sns.barplot(
            y="Valor", x="Frecuencia", data=freq_df,
            ax=ax, color="skyblue", order=freq_df["Valor"]
        )
    else:
        # barras verticais
        sns.barplot(
            x="Valor", y="Frecuencia", data=freq_df,
            ax=ax, color="skyblue", order=freq_df["Valor"]
        )
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")

    # remove bordas
    sns.despine(ax=ax, left=False, bottom=False)

    # adiciona labels
    maxf = freq_df["Frecuencia"].max()
    if len(freq_df) > LIMITE_CATEGORIAS_VERTICAIS:
        for i, row in freq_df.iterrows():
            ax.text(
                row["Frecuencia"] + maxf * 0.01, i,
                f"{row['Frecuencia']} ({row['Porcentaje (%)']:.1f}%)",
                va='center', fontsize=9
            )
    else:
        for i, row in freq_df.iterrows():
            ax.text(
                i, row["Frecuencia"] + maxf * 0.01,
                f"{row['Frecuencia']} ({row['Porcentaje (%)']:.1f}%)",
                ha='center', fontsize=9
            )

    # título multilinha
    main_title = f"Variable: {titulo}"
    if truncado:
        main_title += f" (Top {TOP_N})"
    if area:
        main_title += f"\nÁrea: {area}"
    ax.set_title(main_title, fontsize=11, loc='left')

    # subtítulo
    if subtitulo:
        plt.suptitle(subtitulo, fontsize=12, y=0.96, style='italic', weight='bold')

    ax.set_xlabel("Frecuencia")
    ax.set_ylabel("Categoría")

    # margens e padding
    fig.subplots_adjust(left=0.20, right=0.95, top=0.88, bottom=0.10)
    plt.tight_layout(pad=2.0)

    # salva
    file_name = f"{col}.jpg".replace("/", "_")
    fig.savefig(
        f"../Images/Course/{file_name}",
        format='jpg',
        bbox_inches='tight',
        pad_inches=0.5
    )
    plt.close(fig)

